# SceneXplain: Alt-tagger

In this notebook we will use Jina AI's [SceneXplain](https://scenex.jina.ai) to cycle through a zip file of images and generate an alt-tag for each one. The output will be stored in a CSV file then re-zipped with the images.

For example, if we send the image file: `free-photo-of-red.jpeg`:

![](https://github.com/alexcg1/notebooks/blob/main/scenex/a11y-alt-tags/example/free-photo-of-red.jpeg?raw=true)

We get the output:

```json
{
  "alt_tag": "Rear view of a red classic 'Malibu Classic' car focusing on tail light and chrome bumper"
}
```

We will:

- Unzip the images
- Send them to SceneXplain to get alt-tags
- Write the alt-tags to a CSV file, along with the image filenames
- Re-zip the file

Before proceeding, please sign up for [SceneXplain](https://scenex.jina.ai), create an [API key](https://scenex.jina.ai/api), and paste it below:

In [21]:
SCENEX_SECRET = "<your secret>"

## Notes

### File structure

The zip file must have a flat structure. That is, all image files must be at the top level of the file, e.g:

```
images.zip
├── free-photo-of-a-bowl-of-granola-with-fruit-and-nuts-on-a-wooden-cutting-board.jpeg
├── free-photo-of-alexandrine-parakeet-in-side-view.png
├── free-photo-of-holida-christmas-party-drinks-ornaments.jpeg
├── free-photo-of-leaves-on-the-branch.jpeg
├── free-photo-of-pose-woman-dress-in-the-desert-gold-light-curly-hair.jpeg
├── free-photo-of-red.jpeg
└── pexels-photo-12015253.png
```

### Free plan limitations

Many users of this notebook may be using the free SceneXplain plan, which is limited to eight images per batch. For the sake of this example, we suggest you use [this zipfile of seven images](https://github.com/alexcg1/notebooks/raw/main/scenex/a11y-alt-tags/images.zip). For more complex needs, you may need to either refactor the code in this notebook or switch to a [paid plain](https://scenex.jina.ai/#pricing).

## SceneXplain JSON Store

We will use SceneXplain's JSON Store to pull a predefined schema that is specialized for generating alt-tags. This is just a simple use case for the JSON Store, so be sure to browse and explore what else you can do with it!

Our JSON Schema looks something like this:

```json
{
  "type": "object",
  "properties": {
    "alt_tag": {
      "type": "string",
      "description": "the most concise description possible of the image’s purpose. If the image is purely decorative (e.g. part of the website's design, not content), leave empty. Do not include text like 'this image contains' or 'image depicts'"
    }
  }
}
```

Below we've defined two JSON Schema IDs, one for general alt-tagging, and one specifically targeted for alt-tagging screenshots.

In [1]:
# json_schema = "43IxSufD5tpWYCPkMOj8"  # generic alt-tagger
json_schema = "WJ84ezfnag0zWxgkmDJY"  # screenshot alt-tagger

## User options

Some basic options about input/output/CSV filenames.

In [20]:
# filenames
zip_input = "images.zip"
zip_output = "images_with_alt_tags.zip"
csv_filename = "alt_tags.csv"

## Basic setup

Functions to send generate image lists, populate the data to send to SceneXplain, send that data.

In [3]:
import os
import json
import base64
import http.client
from pprint import pprint
from glob import glob
import csv
import tempfile

In [22]:
# Uncomment this if you use dotenv for secrets

# try:
#     from dotenv import load_dotenv
#     load_dotenv()
#     print("Environment variables loaded from .env")
# except ImportError:
#     os.environ['SCENEX_SECRET'] = SCENEX_SECRET

In [5]:
def extract_zip(zip_file):
    temp_dir = tempfile.mkdtemp()
    with zipfile.ZipFile(zip_input, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

        return temp_dir

In [6]:
def generate_image_list(folder_name, max_count=100):
    filetypes = ['jpg', 'jpeg', 'png']
    image_files = []
    
    for filetype in filetypes:
        image_files.extend(glob(f'{folder_name}/*.{filetype}'))

    return image_files[:max_count]

In [14]:
# SceneX setup and functions

SCENEX_SECRET=os.getenv('SCENEX_SECRET')
features = ["json"]

scenex_headers = {
    "x-api-key": f"token {SCENEX_SECRET}",
    "content-type": "application/json",
}

def image_to_data_uri(file_path):
    with open(file_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
        return f"data:image/png;base64,{encoded_image}"
        
def generate_scenex_data(image_files):
    data = {}
    data['data'] = []

    for file in image_files:
        cid = file.split('/')[-1]
        row = {
            "image": image_to_data_uri(file),
            "features": ["json"],
            "cid": cid,
            "json_schema_id": json_schema
        }

        data['data'].append(row)

    return data

def process_scenex(data):
    connection = http.client.HTTPSConnection("api.scenex.jina.ai")
    connection.request("POST", "/v1/describe", json.dumps(data), scenex_headers)
    response = connection.getresponse()
    response_data = response.read().decode("utf-8")
    
    connection.close()

    return json.loads(response_data)['result']

## Creating our alt-tags

### Unzip images

Unzip the images into a temporary directory which we'll clean up later. For this notebook we'll use an example zipfile of images.

In [19]:
if not os.path.isfile(zip_input):
    print("Downloading images.zip")
    os.system("wget https://github.com/alexcg1/notebooks/raw/main/scenex/a11y-alt-tags/images.zip")

--2023-11-22 15:47:15--  https://github.com/alexcg1/notebooks/raw/main/scenex/a11y-alt-tags/images.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexcg1/notebooks/main/scenex/a11y-alt-tags/images.zip [following]
--2023-11-22 15:47:15--  https://raw.githubusercontent.com/alexcg1/notebooks/main/scenex/a11y-alt-tags/images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4903908 (4.7M) [application/zip]
Saving to: ‘images.zip’

     0K .......... .......... .......... .......... ..........  1% 1.70M 3s
    50K .......... ..........

In [8]:
import zipfile

In [9]:
temp_dir = extract_zip(zip_input)

### Generate data

Let's take the image list and add our SceneXplain options (e.g. JSON Schema ID, image data URI, etc).

In [10]:
image_files = generate_image_list(temp_dir)

In [15]:
scenex_data = generate_scenex_data(image_files)

### Send data to SceneXplain

In [ ]:
scenex_response = process_scenex(scenex_data)

### Write to CSV

In [ ]:
headers = ["filename", "alt-tag"]

In [ ]:
def generate_csv(csv_filename, image_files, scenex_response):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for filename, row in zip(image_files, scenex_response):
            alt_tag = json.loads(row['i18n']['en'])['alt_tag']
            row = [filename, alt_tag]
            writer.writerow(row)

In [ ]:
generate_csv(csv_filename, image_files, scenex_response)

### View results

Let's see how SceneXplain alt-tagged our images:

In [23]:
!cat alt_tags.csv

filename,alt-tag
./images/free-photo-of-leaves-on-the-branch.jpeg,Close-up of tree branch with mix of green and yellow leaves against a background of lush greenery
./images/free-photo-of-a-bowl-of-granola-with-fruit-and-nuts-on-a-wooden-cutting-board.jpeg,Bowl of granola with strawberries and pomegranate seeds on wooden board
./images/free-photo-of-pose-woman-dress-in-the-desert-gold-light-curly-hair.jpeg,Woman with curly hair standing on a beach during sunset or dawn
./images/free-photo-of-holida-christmas-party-drinks-ornaments.jpeg,"Festive Christmas table setting with red tablecloth, two crystal glasses, candy canes, golden ornaments, set against a pink wall and a green curtain"
./images/free-photo-of-red.jpeg,Close-up view of the rear end of a red Malibu Classic car
./images/pexels-photo-12015253.png,Vintage red and white gas pump labeled 'Benzin' on a sidewalk in an urban scene
./images/free-photo-of-alexandrine-parakeet-in-side-view.png,Close-up of green parrot with red beak on 

### Re-zip the files

In [ ]:
with zipfile.ZipFile(zip_output, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

    for foldername, subfolders, filenames in os.walk(temp_dir):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zipf.write(file_path, os.path.relpath(file_path, folder_name))

If you're using Colab you can now right 

### Clean up

In [ ]:
import shutil
os.remove(csv_filename)
shutil.rmtree(temp_dir)